# Ejemplo de Generación de Notas usando `notas_utils`

Este notebook demuestra cómo utilizar el módulo `notas` y `config` para generar datos de notas simulados y calcular los pesos optimizados.

In [13]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import time
import notas as nu

pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('display.max_columns', None)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Definición de items del curso 

La configuración ahora se importa desde el módulo `config.py`.
Puede consultar la documentación de las claves en `config.py`.

In [14]:
# import config as cf
# import config_meccel as cf
import config_astrobio as cf

# Cargamos la configuración desde el módulo externo
config = nu.autoconfigura_items(cf.config_evaluacion)

Autoconfigurando items correlacionados...
 > Quices_Mod1: Ajustado vs Tarea_Mod1 (x0.5)
 > Quices_Mod2: Ajustado vs Quices_Mod1 (x1.2)
 > Tarea_Mod2: Ajustado vs Tarea_Mod1 (x1.2)
 > Quices_Mod3: Ajustado vs Quices_Mod1 (x1.0)
 > Tarea_Mod3: Ajustado vs Tarea_Mod1 (x1.0)


## Generación de Datos

In [15]:
np.random.seed(42)
df_notas = nu.genera_datos(config, N=1000)
print(f"Datos generados: {df_notas.shape}")
df_notas.head()

Datos generados: (1000, 7)


,Quices_Mod1,Tarea_Mod1,Quices_Mod2,Tarea_Mod2,Quices_Mod3,Tarea_Mod3,Examen Final
0,4.12,3.99,4.36,2.39,3.57,2.38,0.00
1,3.14,3.63,4.18,4.76,2.68,4.57,1.77
2,4.00,4.22,2.19,1.40,2.29,2.82,3.08
3,4.15,3.66,3.92,5.00,3.21,3.00,2.65
4,2.10,1.67,4.20,1.14,4.46,1.80,2.79


## Proceso Completo: Generar Encuesta

In [16]:
print(f"Generando notas (100 estudiantes, Rango 3.0-3.5...")

# 0. Forzar nueva aleatoriedad
np.random.seed(int(time.time()))

# 1. Generar conjunto de datos grande
df_survey = nu.genera_datos(config, N=5000)

# 2. Derivar Pesos Sugeridos y Cabeceras
cols_ordered = []
weights_ordered = []
headers = []

reg_weights, w_final = nu.calcular_pesos(config)
reg_items = config['items_normales']

# Construir columnas y pesos COMPLETOS (reg + final) para Promedio
for i, item in enumerate(reg_items):
    w_p = reg_weights[i] * 100.0
    t_orig = item.get('tipo','?')
    header_str = f"{item['nombre']} [{w_p:.1f}%, {t_orig}]"
    cols_ordered.append(item['nombre'])
    weights_ordered.append(reg_weights[i])
    headers.append(header_str)

def_item = config['item_definitorio']
w_f_p = w_final * 100.0
t_orig = def_item.get('tipo','?')
header_str = f"{def_item['nombre']} [{w_f_p:.1f}%, {t_orig}]"
cols_ordered.append(def_item['nombre'])
weights_ordered.append(w_final)
headers.append(header_str)

# 3. Calcular Promedio (Interno para filtrado)
vals = df_survey[cols_ordered].values
avg = np.dot(vals, np.array(weights_ordered))
df_survey['Promedio'] = avg

# 4. Filtrar 3.0 - 3.5
mask_survey = (df_survey['Promedio'] >= 3.0) & (df_survey['Promedio'] <= 3.5)
df_filtered = df_survey[mask_survey].copy()

# 5. Muestrear 100
if len(df_filtered) > 100:
    df_sample = df_filtered.sample(n=100)
else:
    df_sample = df_filtered

print(f"Estudiantes encontrados en rango: {len(df_filtered)}. Seleccionados: {len(df_sample)}")

# 6. Formato Final
df_final = df_sample[cols_ordered].round(1)
df_final.columns = headers

# 7. Columnas de Juicio (Condicional)
col_name = '¿Debería Ganar? (1=Sí, 0=No)'
df_final[col_name] = ""

# 8. Guardar
timestamp = time.strftime("%Y%m%d_%H%M%S")
out_file = f"encuesta_profesores_repro_{timestamp}.xlsx"
df_final.to_excel('./data/' + out_file, index=False)
print(f"Archivo guardado: {out_file}")
df_final.head()

Generando notas (100 estudiantes, Rango 3.0-3.5...
Estudiantes encontrados en rango: 1740. Seleccionados: 100
Archivo guardado: encuesta_profesores_repro_20260201_131051.xlsx


,"Quices_Mod1 [7.5%, facil]","Tarea_Mod1 [15.0%, avanzado]","Quices_Mod2 [9.0%, facil]","Tarea_Mod2 [18.0%, clave]","Quices_Mod3 [7.5%, facil]","Tarea_Mod3 [15.0%, avanzado]","Examen Final [28.0%, examen]","¿Debería Ganar? (1=Sí, 0=No)"
1402,4.20,1.20,3.90,3.30,4.30,3.10,4.00,
3164,3.80,3.80,4.10,4.20,3.20,2.10,1.70,
3098,2.10,3.60,4.10,4.00,4.30,1.70,2.50,
622,4.50,3.50,4.40,0.50,4.30,2.80,3.30,
3901,1.70,3.70,3.40,4.60,3.30,2.30,3.80,
